## 前期准备

alice 和 bob 节点分别启动 ray 集群。

```
ray start --head --node-ip-address="ip" --port="port" --include-dashboard=False --disable-usage-stats
```

屏幕输出中显示 <span style="color: #008A00">Ray runtime started</span>，则说明 Ray 的主节点启动成功。

## 初始化 SecretFlow

alice 和 bob 节点都需要初始化 secretflow，初始化语句只有在 self_party 上有区别。

1. 分别使用 Ray 主节点的 node-ip-address 和 port 填充 sf.init 的 address 参数。
2. parties 中 alice 和 bob 的 address 分别填可以被对方访问的地址，并且选择一个**未被占用的端口。**

In [ ]:
import secretflow as sf
import spu

network_conf = {
    'parties': {
        'alice': {
            'address': 'alice:8080',
        },
        'bob': {
            'address': 'bob:8080',
        },
    },
}

### alice 和 bob 初始化

<span style="color: #BA4081">注意：下面初始化语句是分别只在对应的 alice 或 bob 节点执行的。除了初始化语句，后面所有代码，会被同时下发到 alice 和 bob 节点执行。</span>

In [ ]:
sf.shutdown()
sf.init(
    address='127.0.0.1:6379',
    cluster_config={**network_conf, 'self_party': 'alice'},
    log_to_driver=True,
)

In [ ]:
sf.shutdown()
sf.init(
    address='127.0.0.1:6379',
    cluster_config={**network_conf, 'self_party': 'bob'},
    log_to_driver=True,
)

## 初始化 SPU

1. alice 的 address 请填写可以被 bob 访通的地址，并且选择一个**未被占用的端口** ，注意不要和 Ray 端口冲突。
2. alice 的 listen_addr 可以和 alice address 里的端口一样。
3. bob 的 address 请填写可以被 alice 访通的地址，并且选择一个**未被占用的端口** ，注意不要和 Ray 端口冲突。
4. bob 的 listen_addr 可以和 bob address 里的端口一样。

In [ ]:
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu_conf = {
    "nodes": [
        {
            "party": "alice",
            "address": "alice:8081",
            "listen_addr": "alice:8081"
        },
        {
            "party": "bob",
            "address": "bob:8081",
            "listen_addr": "alice:8081"
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}
spu = sf.SPU(cluster_def=spu_conf)

## 隐私求交

<span style="color: rgb(50, 50, 50)">提供 </span>`psi_csv`<span style="color: rgb(50, 50, 50)"> 函数， </span>`psi_csv`<span style="color: rgb(50, 50, 50)"> 将 csv 文件作为输入，并在求交后生成 csv 文件。默认协议为 </span>[**KKRT**](https://eprint.iacr.org/2016/799.pdf)<span style="color: rgb(50, 50, 50)">。</span>

In [ ]:
import os

current_dir = os.getcwd()

input_path = {alice: f'{current_dir}/iris_alice.csv', bob: f'{current_dir}/iris_bob.csv'}
output_path = {alice: f'{current_dir}/iris_alice_psi.csv', bob: f'{current_dir}/iris_alice_psi.csv'}
spu.psi_csv('uid', input_path, output_path, 'alice')